# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
The output of the model is "need intervention" or "not need intervention" is a binary variable, therefore the model should be a binary classification 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
#getPrintOptions = np.get_printoptions()
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!\n"

Student data read successfully!



### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = (student_data['passed'] == 'yes').sum()

# TODO: Calculate failing students
n_failed = n_students - n_passed
 
# TODO: Calculate graduation rate
grad_rate = 100.0*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all0 = student_data[feature_cols]
y_all0 = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
X_all0.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4


### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

**Binary features:**
- `school` : student's school (binary: "GP" or "MS")
- `sex` : student's sex (binary: "F" - female or "M" - male)
- `address` : student's home address type (binary: "U" - urban or "R" - rural)
- `famsize` : family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)
- `schoolsup` : extra educational support (binary: yes or no)
- `famsup` : family educational support (binary: yes or no)
- `paid` : extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
- `activities` : extra-curricular activities (binary: yes or no)
- `nursery` : attended nursery school (binary: yes or no)
- `higher` : wants to take higher education (binary: yes or no)
- `internet` : Internet access at home (binary: yes or no)
- `romantic` : with a romantic relationship (binary: yes or no)


**Nominal/categorical features:**
- `Pstatus` : parent's cohabitation status (binary: "T" - living together or "A" - apart)
- `Medu` : mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)
- `Fedu` : father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)
- `Mjob` : mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
- `Fjob` : father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
- `reason` : reason to choose this school (nominal: close to "home", school "reputation", "course" preference or "other")
- `guardian` : student's guardian (nominal: "mother", "father" or "other")
- `traveltime` : home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
- `studytime` : weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
- `failures` : number of past class failures (numeric: n if 1<=n<3, else 4)
- `famrel` : quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
- `freetime` : free time after school (numeric: from 1 - very low to 5 - very high)
- `goout` : going out with friends (numeric: from 1 - very low to 5 - very high)
- `Dalc` : workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
- `Walc` : weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
- `health` : current health status (numeric: from 1 - very bad to 5 - very good)

**Continous features:**
- `age` : student's age (numeric: from 15 to 22)
- `absences` : number of school absences (numeric: from 0 to 93)

**Binary target:**
- `passed` : did the student pass the final exam (binary: yes or no)

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output


def preprocess_features_v2(Xdf):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame()
    
    df_columns = Xdf.columns # all features
    
    #after reading "Data" section in the `readme.md` file the continious features are 
    df_continuos = np.array(['age','absences'])
    df_category_cols = np.setdiff1d(df_columns,df_continuos) #all category features
    
    # Investigate each category feature column for the data
    for col in df_category_cols:
        col_set = Xdf[col].unique()
        if len(col_set) <= 2:
            #transform binary features in 1/0
            output[col+'_'+col_set[0]] = Xdf[col].replace(col_set,[1,0])
        else:
            #transform category features in dummy variables == one hot encoding?
            one_hot = pd.get_dummies(Xdf[col], prefix=col)

            if output.empty: 
                output = one_hot # cannot join with empty dataFrame
            else:
                output = output.join(one_hot)
    
    #normalize continous features (f - f.mean)/f.std
    for col in df_continuos:
        output[col] = (Xdf[col]-Xdf[col].mean())/Xdf[col].std()
        
    return output

In [5]:
X_all = preprocess_features_v2(X_all0)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
X_all.head()

Processed feature columns (84 total features):
['Dalc_1', 'Dalc_2', 'Dalc_3', 'Dalc_4', 'Dalc_5', 'Fedu_0', 'Fedu_1', 'Fedu_2', 'Fedu_3', 'Fedu_4', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'Medu_0', 'Medu_1', 'Medu_2', 'Medu_3', 'Medu_4', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Pstatus_A', 'Walc_1', 'Walc_2', 'Walc_3', 'Walc_4', 'Walc_5', 'activities_no', 'address_U', 'failures_0', 'failures_1', 'failures_2', 'failures_3', 'famrel_1', 'famrel_2', 'famrel_3', 'famrel_4', 'famrel_5', 'famsize_GT3', 'famsup_no', 'freetime_1', 'freetime_2', 'freetime_3', 'freetime_4', 'freetime_5', 'goout_1', 'goout_2', 'goout_3', 'goout_4', 'goout_5', 'guardian_father', 'guardian_mother', 'guardian_other', 'health_1', 'health_2', 'health_3', 'health_4', 'health_5', 'higher_yes', 'internet_no', 'nursery_yes', 'paid_no', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'romantic_no', 'school_GP', 'schoolsup_yes',

,Dalc_1,Dalc_2,Dalc_3,Dalc_4,Dalc_5,Fedu_0,Fedu_1,Fedu_2,Fedu_3,Fedu_4,...,studytime_1,studytime_2,studytime_3,studytime_4,traveltime_1,traveltime_2,traveltime_3,traveltime_4,age,absences
0,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,1.021751,0.036378
1,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0.238078,-0.213525
2,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,-1.329268,0.536185
3,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,-1.329268,-0.463428
4,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,-0.545595,-0.213525


In [6]:
y_all = y_all0.replace(['yes', 'no'], [1, 0])

### Check for correlated features ....

In [7]:
#there are quite few correlated features
cov = X_all.corr().as_matrix()

rlim = [-0.2, 0.2] # limits for the pearson r

covrlim = np.transpose(np.where(((cov<=rlim[0]) | (cov>=rlim[1])) & (cov !=1.0)))
print covrlim[:20]
covmval = cov[covrlim[:,0],covrlim[:,1]]
covmval[:20]

[[ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 0 26]
 [ 0 29]
 [ 0 30]
 [ 0 73]
 [ 1  0]
 [ 1 26]
 [ 1 28]
 [ 1 29]
 [ 2  0]
 [ 2 26]
 [ 2 29]
 [ 3  0]
 [ 3 30]
 [ 4  0]
 [ 4 30]
 [ 4 53]]


array([-0.73728721, -0.40425421, -0.23254589, -0.23254589,  0.5051952 ,
       -0.33956159, -0.33465228,  0.23940931, -0.73728721, -0.36756246,
        0.22181395,  0.23708982, -0.40425421, -0.20881758,  0.23270137,
       -0.23254589,  0.22226502, -0.23254589,  0.55281729,  0.23855291])

In [8]:
# PCA to reduce the number of features - eliminates correlated features
pca = PCA()
X_pca = pca.fit_transform(X_all)
# take the first k of PC
k = 10
X_all_pca = X_pca[:,:k]

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [9]:
# TODO: Import any additional functionality you may need here
from sklearn.utils import shuffle

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
RANDOM_STATE = 63
X_all, y_all = shuffle(X_all, y_all, random_state = RANDOM_STATE)
X_all_pca, y_all = shuffle(X_all_pca, y_all, random_state = RANDOM_STATE)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

X_pca_train = X_all_pca[:num_train]
X_pca_test = X_all_pca[num_train:]

y_train = y_all[:num_train]
y_test = y_all[num_train:]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print "PCA Training set has {} samples.".format(X_pca_train.shape[0])
print "PCA Testing set has {} samples.".format(X_pca_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.
PCA Training set has 300 samples.
PCA Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

### 1. Support Vector Machines

**- Applications **of SVM for oil and gas exploration seems to be quite effective compared to other data mining algorithms. [refrence](https://www.researchgate.net/publication/243190076 )
    
**- Strength **
  - prediction accuracy is generally high 
  - robust  works when training examples contain errors
  - fast evaluation of the learned target function

**- Weaknesses ** (https://en.wikipedia.org/wiki/Support_vector_machine)
  - Requires full labeling of input data
  - Uncalibrated class membership probabilities
  - The SVM is only directly applicable for two-class tasks. Therefore, algorithms that reduce the multi-class task to several binary problems have to be applied; see the multi-class SVM section.
  - Parameters of a solved model are difficult to interpret.

**-SVM** is a good candidate as our problem is binary classification with less than 100k data points

### 2. Nearest Neighbors classifier

**- Application ** of k-NN search is the [Recommender Systems](https://en.wikipedia.org/wiki/Recommender_system) . If you know a user likes a particular item, then you can recommend similar items for them. To find similar items, you compare the set of users who like each item—if a similar set of users like two different items, then the items themselves are probably similar!

**-Strength **
  - The cost of the learning process is zero
  - No assumptions about the characteristics of the concepts to learn have to be done
  - Complex concepts can be learned by local approximation using simple procedure
 
**-Weaknesses **
  - The model can not be interpreted (there is no description of the learned concepts)
  - It is computationally expensive to find the k nearest neighbours when the dataset is very large
  - Performance depends on the number of dimensions that we have

**- kNN ** especially with the kernel trick can classify our data 


### 3. Bagging ensemble classifier

**- Application ** of bagging ensemble classifier to improve the performance of network intrusion detection systems. [reference](http://link.springer.com/chapter/10.1007/978-3-642-31537-4_46)

**-Strength **
  - combining models (weak learners) improves model accuracy on average as the number of models in the combination increased.
  -  more resilient to noise 
  -  is very useful for large and high-dimensional data

**-Weaknesses **
  - difficult to interpret (a linear combination of decision trees is much harder to interpret than a single tree)
  - using Bagging with a biased model is not going to always work
  

**- Bagging ** classifier combined with kNN achieved a good prediction for our dataset



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    # Print the results
    #print "Trained model in {:.4f} seconds".format(end - start)
    return end-start

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    #print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label=1), end-start


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [11]:
# Naive Bayes default settings - KIND of Linear classifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

# K -neighbors classifier
from sklearn.neighbors import KNeighborsClassifier

#Decision Trees
from sklearn.tree import DecisionTreeClassifier

#Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)

##Bagging##
from sklearn.ensemble import BaggingClassifier
##AdaBoost##
from sklearn.ensemble import AdaBoostClassifier
##Random Forest##
from sklearn.ensemble import RandomForestClassifier

##Gradient Boosting##
from sklearn.ensemble import GradientBoostingClassifier

#Stochastic Gradient Descent (SGDC)
from sklearn.linear_model import SGDClassifier

#Support Vector Machines (SVM)
from sklearn.svm import SVC

#Logistic Regression
from sklearn.linear_model import LogisticRegression

In [12]:
def train_predict_all(clfs, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    df = pd.DataFrame(columns=['Classifier','f1-test','f1-train','train-time','predict-train-time','predict-test-time'])
    i = 0
    for item in clfs:
        p = item['param'] #the parms for the classifier
        c = item['clf'](**p) # the current classifier
        c_name = c.__class__.__name__ #the classifier name
        train_time = train_classifier(c, X_train, y_train) 
        f1_train, t_predict_train = predict_labels(c, X_train, y_train)
        f1_test, t_predict_test = predict_labels(c, X_test, y_test)

        df.loc[i] =[c_name, f1_test,f1_train,train_time,t_predict_train,t_predict_test]
        #rint c_name, f1_test,f1_train,train_time,t_predict_train,t_predict_test
        i +=1
        
    #sort classifiers according to the f1-test score
    dfs = df.sort_values(by=['f1-test'], ascending=False)
    dfss = dfs.reset_index().drop('index',axis=1)
    return dfss

### For comparision I used the following clasifiers with their deafault parameters

In [13]:
classifiers = [
    {
        'clf':BernoulliNB,
        'param':{}
    },{
        'clf':GaussianNB,
        'param':{}
    },{
        'clf':KNeighborsClassifier,
        'param':{}
    },{
        'clf':DecisionTreeClassifier,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':BaggingClassifier,
        'param':{'base_estimator':KNeighborsClassifier(),'max_samples':0.5, 'max_features':0.5, 'random_state':RANDOM_STATE}
    },{
        'clf':AdaBoostClassifier,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':RandomForestClassifier,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':GradientBoostingClassifier,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':SGDClassifier,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':SVC,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':LogisticRegression,
        'param':{'random_state':RANDOM_STATE}
    }
] 

### Results considering all features

In [14]:
df = train_predict_all(classifiers, X_train, y_train, X_test, y_test)
df

,Classifier,f1-test,f1-train,train-time,predict-train-time,predict-test-time
0,SVC,0.782051,0.809524,0.006543,0.005001,0.001636
1,BaggingClassifier,0.760000,0.821355,0.013030,0.026548,0.012403
2,KNeighborsClassifier,0.741259,0.821429,0.000592,0.009317,0.003501
3,GradientBoostingClassifier,0.704225,0.983133,0.068863,0.000672,0.000300
4,AdaBoostClassifier,0.671533,0.853828,0.068245,0.004133,0.002992
5,BernoulliNB,0.656489,0.786070,0.002146,0.000630,0.000264
6,LogisticRegression,0.647059,0.835616,0.001398,0.000135,0.000101
7,DecisionTreeClassifier,0.639344,1.000000,0.001601,0.000184,0.000129
8,RandomForestClassifier,0.635659,0.990148,0.017938,0.003963,0.003902
9,SGDClassifier,0.578512,0.712329,0.000666,0.000176,0.000121


### Results considering the first 10 PCA features

In [15]:
df_pca = train_predict_all(classifiers, X_pca_train, y_train, X_pca_test, y_test)
df_pca

,Classifier,f1-test,f1-train,train-time,predict-train-time,predict-test-time
0,BaggingClassifier,0.784314,0.817814,0.009441,0.010704,0.007535
1,LogisticRegression,0.784314,0.804928,0.000463,0.000046,0.000033
2,SVC,0.782051,0.809524,0.002880,0.001671,0.000560
3,BernoulliNB,0.776316,0.807377,0.000597,0.000120,0.000090
4,GaussianNB,0.743243,0.792208,0.000387,0.000138,0.000092
5,DecisionTreeClassifier,0.737705,1.000000,0.001465,0.000070,0.000045
6,KNeighborsClassifier,0.736111,0.824834,0.000415,0.004499,0.001086
7,AdaBoostClassifier,0.735294,0.886311,0.070511,0.003670,0.002775
8,GradientBoostingClassifier,0.731034,0.973747,0.036777,0.000352,0.000163
9,SGDClassifier,0.731034,0.750562,0.000333,0.000043,0.000045


In [16]:
# TODO: Initialize the three models with sklearn default parameters

models = [
    {
        'clf':SVC,
        'param':{'random_state':RANDOM_STATE}
    },{
        'clf':BaggingClassifier,
        'param':{'base_estimator':KNeighborsClassifier(),'max_samples':0.5, 'max_features':0.5,'random_state':RANDOM_STATE }
    },{
        'clf':KNeighborsClassifier,
        'param':{}
    }
] 

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

In [17]:
df_100 = train_predict_all(models, X_train_100, y_train_100, X_test, y_test)
df_100

,Classifier,f1-test,f1-train,train-time,predict-train-time,predict-test-time
0,SVC,0.782051,0.850575,0.001351,0.000734,0.000640
1,BaggingClassifier,0.766234,0.855491,0.009768,0.009045,0.008454
2,KNeighborsClassifier,0.731034,0.883436,0.000310,0.001441,0.001403


In [18]:
df_200 = train_predict_all(models, X_train_200, y_train_200, X_test, y_test)
df_200

,Classifier,f1-test,f1-train,train-time,predict-train-time,predict-test-time
0,SVC,0.782051,0.809524,0.003720,0.002473,0.001248
1,BaggingClassifier,0.751678,0.813456,0.010521,0.015176,0.009986
2,KNeighborsClassifier,0.724638,0.832215,0.000403,0.003892,0.002194


In [19]:
df_300 = train_predict_all(models, X_train_300, y_train_300, X_test, y_test)
df_300

,Classifier,f1-test,f1-train,train-time,predict-train-time,predict-test-time
0,SVC,0.782051,0.809524,0.007329,0.006024,0.001722
1,BaggingClassifier,0.760000,0.821355,0.010395,0.027040,0.012360
2,KNeighborsClassifier,0.741259,0.821429,0.000510,0.008777,0.003256


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.001124               | 0.000637               |   0.850575       |   0.782051      |
| 200               |  0.004117 	          | 0.001263               |   0.809524       |   0.782051      |
| 300               |  0.007259               | 0.001696               |   0.809524 	  |   0.782051      |

** Classifer 2 - Bagging**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.000289 	          |   0.001374             |   0.855491       |   0.766234      |
| 200               |  0.010534               |   0.009759             |   0.813456       |   0.751678      |
| 300               |  0.009705               |   0.012190             |   0.821355       |   0.760000      |

** Classifer 3 - kNN**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.000289              |   0.001374             |    0.883436      |   0.731034      |
| 200               |   0.000366              |   0.002147             |    0.832215      |   0.724638      |
| 300               |   0.000476 	          |   0.003182             |    0.821429 	  |   0.741259      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
 Considering the above results I choose SVC for the following reasons:
 - achieved the highest F1 score
 - need less data to train then the other models (F1-score for the test data is the same for 100,200, and 300 point training data
 - the traning time and prediction time si the second best after kNN

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

SVM is about to find a decision boundary that separates two classes (eg. graduated -green vs. not graduated-red)  as shown on the illustration below

![schematic](http://www.statsoft.com/textbook/graphics/SVMIntro1.gif)
In order to find this boundary the model is trained with data that was properly preporecessed. Furthermore, the model's parameters were tuned to produce the best decision boundary that will allow more accurate predictions. Once the model is trained and we are satisfaied with its performance, it can be used to predict if the a student is likely to gaduate or not based on the collected data. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [20]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
#(estimator, param_grid, scoring=None, fit_params=None, n_jobs=1, iid=True, refit=True, cv=None, verbose=0, 
#pre_dispatch='2*n_jobs', error_score='raise')
from sklearn.preprocessing import normalize
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
#(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, 
#cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=None, random_state=None)

X_train = normalize(X_train)
X_test = normalize(X_test)

C_range = np.logspace(-3, 3, 13)
gamma_range = np.logspace(-4, 2, 13)
kernel =('linear','rbf','sigmoid')
parameters = dict(gamma=gamma_range, C=C_range, kernel=kernel)


# TODO: Initialize the classifier
clf = SVC(random_state=RANDOM_STATE)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, cv=5, scoring=f1_scorer, verbose=1, n_jobs=8)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)[0])
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)[0])
print '\nBest model\n',clf

Fitting 5 folds for each of 507 candidates, totalling 2535 fits


[Parallel(n_jobs=8)]: Done 808 tasks      | elapsed:    4.6s
[Parallel(n_jobs=8)]: Done 2520 out of 2535 | elapsed:   24.9s remaining:    0.1s


Tuned model has a training F1 score of 0.8040.
Tuned model has a testing F1 score of 0.7821.

Best model
SVC(C=0.31622776601683794, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=100.0, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=63, shrinking=True,
  tol=0.001, verbose=False)


[Parallel(n_jobs=8)]: Done 2535 out of 2535 | elapsed:   25.5s finished


### Considering the imabalnced dataset
I expected a better result using the imlearn librery but it seems that the result is not that good!?

In [21]:
from imblearn import over_sampling as os
from imblearn import pipeline as pl
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced

In [22]:
pipeline = pl.make_pipeline(os.SMOTE(random_state=RANDOM_STATE),
                            SVC(C=0.3162,gamma=100.0,kernel='sigmoid',random_state=RANDOM_STATE))
# Split the data
X_train_im, X_test_im, y_train_im, y_test_im = train_test_split(X_all, y_all,test_size=0.24,random_state=RANDOM_STATE)

X_train_im_n = normalize(X_train_im)
X_test_im_n = normalize(X_test_im)

# Train the classifier with balancing
pipeline.fit(X_train_im_n, y_train_im)

# Test the classifier and get the prediction
y_pred_bal = pipeline.predict(X_test_im_n)

# Show the classification report
print(classification_report_imbalanced(y_test_im, y_pred_bal))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.20      0.03      0.94      0.06      0.37      0.13        29
          1       0.69      0.94      0.03      0.79      0.37      0.14        66

avg / total       0.54      0.66      0.31      0.57      0.37      0.14        95



### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final F<sub>1</sub> is 0.7821 that is equal to the untuned model. It seems that for this data set the optimal hyperparameters are the default ones

However, if I consider the fact the many of the features are correlated and using PCA the winner are the Bagging and Logistic classifier 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.